In [2]:
# Python 3.6, Numpy 1.16.2, OpenCV, scikit-image, tensorflow 1.3
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import vgg19_fastmal as vgg19
import utils
from sklearn.model_selection import train_test_split
import os
import csv
import numpy as np
import pandas as pd
from random import random, shuffle

def majority_voting(votes):
    return max(votes,key=votes.count)

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
# make the training and testing csv file
# read the csv file as a dataframe
data = pd.read_csv("../../lib/NPM1_slide_patch_num.csv",header=None)
# set the first row as the column name
data.columns = data.iloc[0]
# random select 20% of the data replace the data in the dataframe
data = data.sample(frac=0.2,replace=True)
data = data.reset_index(drop=True)
# add A to the first column
data.iloc[:,0] = 'A' + data.iloc[:,0].astype(str)
# split the dataframe into training and testing by 80% and 20%
df_train , df_test = train_test_split (data,test_size=0.2,random_state=100)
# df_train

In [5]:
df_test

,Slide,Target,patches_number
88,A957,0,2453
80,A337,0,3398
50,A488,0,8632
28,A518,1,6641
95,A1066,0,3433
74,A811,0,1239
26,A770,0,1945
32,A507,0,986
33,A640,0,3712
25,A721,1,1067


In [6]:
csv_slide_ids = df_train['Slide'].tolist()
print(len(csv_slide_ids))
csv_labels =np.array(df_train['Target']).astype(np.uint8)
onehot_labels = np.zeros(shape=(csv_labels.shape[0], 2), dtype=np.float32)
onehot_labels[csv_labels==0,0]=1
onehot_labels[csv_labels==1,1]=1

92


In [7]:
test_csv_slide_ids = df_test['Slide'].tolist()
print(len(test_csv_slide_ids))
test_csv_labels =np.array(df_test['Target']).astype(np.uint8)

23


In [8]:
#load the data path
dataset = "/home/exon_storage1/aml_slide/single_cell_image/"
dataset2 = None
subdirsAll = os.listdir(dataset)

selected_slides = []
selected_labels=[]
selected_onehot_labels = []

test_slides = []
test_labels=[]
test_onehot_labels = []

for index, slide in enumerate(csv_slide_ids):
    if slide in subdirsAll:
        # print(f'{slide} has label {csv_labels[index]} or {onehot_labels[index,:]}')
        slide_path = os.path.join(dataset, slide)
        selected_slides.append(slide_path) 
        selected_onehot_labels.append(onehot_labels[index,:])
        selected_labels.append(csv_labels[index])

for index, slide in enumerate(test_csv_slide_ids):
    if slide in subdirsAll:
        slide_path = os.path.join(dataset, slide)
        # print(slide, ' has label ', test_csv_labels[index])
        test_slides.append(slide_path) 
        test_labels.append(test_csv_labels[index])  

selected_labels=np.array(selected_labels)
positive_ids = np.nonzero(selected_labels)[0]
negative_ids = np.nonzero(1-selected_labels)[0]

# print(selected_slides[0])
print("Negative train:", np.sum(1-selected_labels))
print("Positive train:", np.sum(selected_labels))

print("Negative test:", np.sum(1-np.array(test_labels)))
print("Positive test:", np.sum(np.array(test_labels)))

Negative train: 77
Positive train: 15
Negative test: 17
Positive test: 6


In [9]:
IMSIZE = 128
num_labels=2
num_steps = 0
batch_size= 1
rpt_interval=100
min_nb_images = 1
save_dir = "../output/"
with tf.Session() as sess:
    images = tf.placeholder(tf.float32, [None, IMSIZE, IMSIZE, 3])
    true_out = tf.placeholder(tf.float32, [None, 2])
    train_mode = tf.placeholder(tf.bool)

    vgg = vgg19.Vgg19("../model/vgg19.npy", imsize=IMSIZE)
    vgg.build_avg_pool(images, train_mode=train_mode)
    focal_loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=vgg.new_fc8, labels=true_out)) 
    train = tf.train.GradientDescentOptimizer(0.0003).minimize(focal_loss)
    sess.run(tf.global_variables_initializer())
    step_pos=0
    step_neg=0
    for step in range(1, num_steps+1):
        try:  
            offset = (step * batch_size) % (len(selected_onehot_labels) - batch_size)
            
            if step%2==0:
                sl_id = positive_ids[step_pos % (positive_ids.shape[0])]
                step_pos=step_pos+1
            else:
                sl_id = negative_ids[step_neg % (negative_ids.shape[0])]
                step_neg=step_neg+1

            slides=utils.load_folder_random(selected_slides[sl_id], max_no_img=100, crop_size=IMSIZE)
            if len(slides)>min_nb_images:
                slides = np.array(slides)
                labels = np.reshape(selected_onehot_labels[sl_id], (batch_size, num_labels))
                _,l = sess.run([train, focal_loss], feed_dict={images: slides, true_out: labels, train_mode: True})
            if (step % rpt_interval == 0):
                print('Minibatch loss at step %d: %f' % (step, l))     
            if (step % 5000 == 0):   
                save_path = os.path.join(save_dir,dataset.split(os.path.sep)[-3]+str(step)+"sickle_max_pool_ce_vgg19_model.npy")
                vgg.save_npy(sess, save_path)
                print("Model saved in file: %s" % save_path)    
        except IOError as e:
            print('Could not read:', selected_slides[offset], ':', e, '- it\'s ok, skipping.')        
# test classification again, should have a higher probability about tiger
    prediction_csv = dataset.split(os.path.sep)[-2]+'_mv_all_normal_max_pool2.csv' 
    header=['Slide-Id', 'True', 'Predicted']
    predictionFile= open(os.path.join("../output/", prediction_csv),'w')  
    wr = csv.writer(predictionFile, dialect='excel')
    wr.writerow(header)    
    predicted_classif = np.zeros(len(test_slides))
    true_classif = np.zeros(len(test_slides))
    for tt in range(len(test_slides)):
        
        classify_voting=[]
        probs=[]
        for run in range(1):
        #tslides = utils.load_folder(test_slides[tt], crop_size=64)
            tslides=utils.load_folder_random(test_slides[tt], max_no_img=4, crop_size=IMSIZE)
        
            tslide_id = test_slides[tt].split(os.path.sep)[-1]
            malaria_classif=0
            if len(tslides)>min_nb_images:
                tslides = np.array(tslides)

                prob = sess.run(vgg.new_prob, feed_dict={images: tslides, train_mode: False})
                #print(prob)
                classify_voting.append(np.argmax(prob))
                probs.append(prob[0,1])
        sel_id = list(test_csv_slide_ids).index(tslide_id)
        true_classif[tt] = test_csv_labels[sel_id]
        predicted_classif[tt]=majority_voting(classify_voting)
        #predicted_classif[tt]=max(classify_voting)
        wr.writerow([tslide_id, true_classif[tt], predicted_classif[tt], np.mean(probs)])  
        print(tslide_id, true_classif[tt], predicted_classif[tt], np.mean(probs))
        
        
        
    

    overall_accuracy=np.mean(true_classif==predicted_classif)    
    print('Overall accuracy', overall_accuracy)
    true_pos = true_classif[true_classif==1]
    pred_pos = predicted_classif[true_classif==1]
    positive_accuracy=np.mean(true_pos==pred_pos)
    print('Postive accuracy', positive_accuracy)
    true_neg = true_classif[true_classif==0]    
    pred_neg = predicted_classif[true_classif==0]
    negative_accuracy=np.mean(true_neg==pred_neg)
    print('Negative accuracy', negative_accuracy)   

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

A957 0.0 0.0 0.4998438
A337 0.0 0.0 0.49983564
A488 0.0 0.0 0.49987054
A518 1.0 0.0 0.49984232
A1066 0.0 0.0 0.49986795
A811 0.0 0.0 0.49986798
A770 0.0 0.0 0.4998795
A507 0.0 0.0 0.49984387
A640 0.0 0.0 0.49982235
A721 1.0 0.0 0.49985737
A536 0.0 0.0 0.49981573
A636 0.0 0.0 0.49986243
A292 0.0 0.0 0.49986064
A642 0.0 0.0 0.49987477
A514 0.0 0.0 0.4998551
A427 0.0 0.0 0.49984822
A514 0.0 0.0 0.4998495
A993 0.0 0.0 0.49984995
A457 1.0 0.0 0.49986005
A311 0.0 0.0 0.4998342
A960 1.0 0.0 0.49985152
A768 1.0 0.0 0.49984136
A946 1.0 0.0 0.49984246
Overall accuracy 0.7391304347826086
Postive accuracy 0.0
Negative accuracy 1.0
